In [0]:

%doctest_mode

In [0]:
import numpy as np
import pandas as pd
import pandas_datareader as web
import datetime

# import a file that contains a summary statistics function

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
# Set color style
plt.style.use('seaborn-dark')
plt.style.use("tableau-colorblind10")

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [0]:


# Import INDIABULLS data
ind = web.get_data_yahoo('IBULHSGFIN.NS', 
                          start=datetime.datetime(2013, 7, 23), 
                          end=datetime.datetime(2020, 3, 2))
ind.tail()

In [0]:
# 100 day moving average for the adjusted close, adding it to the DataFrame
ind_100MA = ind["Adj Close"].rolling(window=100).mean()
ind["100MA"] = ind_100MA
ind.head()

In [0]:

bull = ind[["Adj Close", "100MA"]]
bull.dropna(inplace=True)
bull.head()

In [0]:
Position = []
for i in range(0,bull.shape[0]):
    if bull["Adj Close"].iloc[i] > bull["100MA"].iloc[i]:
        Position.append("Long")
    else:
        Position.append("Short")

bull["Position"] = Position
bull.tail()

In [0]:
# return as percentage change from yesterday to today
bull["return"] = (bull["Adj Close"] - bull["Adj Close"].shift())/bull["Adj Close"].shift()
bull.dropna(inplace=True)
bull.tail()

In [0]:
# Start with initial amount equal to the value of Apple at time 1
# If the position of yesterday was long, then we add the gain from today to tomorrow
# by multiplying (1+r)
# if the position was short, we subtract the gain or loss by multyplying with (1+r).


LongShort = [0]*bull.shape[0]
LongHold = [0]*bull.shape[0]

LongShort[0] = bull["Adj Close"].iloc[0]
LongShort[1] = bull["Adj Close"].iloc[0]
LongHold[0] = bull["Adj Close"].iloc[0]    
LongHold[1] = bull["Adj Close"].iloc[0]    

    
for i in range(0, bull.shape[0]-2):
    if bull["Position"].iloc[i] == "Long":
        LongShort[i+2] = LongShort[i+1]*(1+bull["return"][i+2])
    else:
        LongShort[i+2] = LongShort[i+1]/(1+bull["return"][i+2])
        
for i in range(0, bull.shape[0]-2):
    if bull["Position"].iloc[i] == "Long":
        LongHold[i+2] = LongHold[i+1]*(1+bull["return"][i+2])
    else:
        LongHold[i+2] = LongHold[i+1]

bull["LongShort"] = LongShort
bull["LongHold"] = LongHold
bull.drop(bull.tail(1).index,inplace=True)
bull.head()

In [0]:

fig = plt.figure(figsize=(20,12))
ax1 = plt.plot(bull["Adj Close"])
ax1 = plt.plot(bull["100MA"])
ax1 = plt.plot(bull["LongShort"], color="green")
ax1 = plt.plot(bull["LongHold"], color="brown")
ax1 = plt.title("INDIABULLS daily ajusted close and strategy from 2009 through 2018", fontsize=18)
ax1 = plt.xlabel("Date", fontsize=18)
ax1 = plt.ylabel("Price", fontsize=18)
ax1 = plt.legend(["Price", "100 day SMA", "SMA crossover long/short strategy",
                  "SMA crossover long/cash strategy"],prop={"size":22}, loc="upper left")
plt.grid(True)
plt.show()

# Copy the previous code and replace the SMA with the **EMA**

In [0]:
# Import INDIABULLS data
ind = web.get_data_yahoo('IBULHSGFIN.NS', 
                          start=datetime.datetime(2013, 7, 23), 
                          end=datetime.datetime(2020, 3, 2))

ind_100EMA = ind["Adj Close"].ewm(100).mean()
ind["100EMA"] = ind_100EMA

bull = ind[["Adj Close", "100EMA"]]
bull.dropna(inplace=True)

Position = []
for i in range(0,bull.shape[0]):
    if bull["Adj Close"].iloc[i] > bull["100EMA"].iloc[i]:
        Position.append("Long")
    else:
        Position.append("Short")

bull["Position"] = Position

bull["return"] = (bull["Adj Close"] - bull["Adj Close"].shift())/bull["Adj Close"].shift()
bull.dropna(inplace=True)

LongShort = [0]*bull.shape[0]
LongHold = [0]*bull.shape[0]

LongShort[0] = bull["Adj Close"].iloc[0]
LongShort[1] = bull["Adj Close"].iloc[0]
LongHold[0] = bull["Adj Close"].iloc[0]    
LongHold[1] = bull["Adj Close"].iloc[0]    

    
for i in range(0, bull.shape[0]-2):
    if bull["Position"].iloc[i] == "Long":
        LongShort[i+2] = LongShort[i+1]*(1+bull["return"][i+2])
    else:
        LongShort[i+2] = LongShort[i+1]/(1+bull["return"][i+2])
        
for i in range(0, bull.shape[0]-2):
    if bull["Position"].iloc[i] == "Long":
        LongHold[i+2] = LongHold[i+1]*(1+bull["return"][i+2])
    else:
        LongHold[i+2] = LongHold[i+1]

bull["LongShort"] = LongShort
bull["LongHold"] = LongHold
bull.drop(bull.tail(1).index,inplace=True)

fig = plt.figure(figsize=(18,12))
ax1 = plt.plot(bull["Adj Close"])
ax1 = plt.plot(bull["100EMA"])
ax1 = plt.plot(bull["LongShort"], color="green")
ax1 = plt.plot(bull["LongHold"], color="brown")
ax1 = plt.title("INDIABULLS daily ajusted close and strategy from 2009 through 2018", fontsize=18)
ax1 = plt.xlabel("Date", fontsize=18)
ax1 = plt.ylabel("Price", fontsize=18)
ax1 = plt.legend(["Price", "100 day EMA", "EMA crossover long/short strategy", "EMA crossover long/cash strategy"],prop={"size":22}, loc="upper left")
plt.grid(True)
plt.show()

# MACD
Moving average convergence divergence compares a short term moving average with a longer term. When the shorter crosses over the longer one, the

In [0]:
# Import INDIABULLS data
ind = web.get_data_yahoo('IBULHSGFIN.NS', 
                          start=datetime.datetime(2013, 7, 23), 
                          end=datetime.datetime(2020, 3, 2))

ind_21EMA = ind["Adj Close"].ewm(21).mean()
ind_126EMA = ind["Adj Close"].ewm(126).mean()
ind["21EMA"] = ind_21EMA
ind["126EMA"] = ind_126EMA

bull = ind[["Adj Close", "21EMA", "126EMA"]]
bull.dropna(inplace=True)

Position = []
for i in range(0,bull.shape[0]):
    if bull["21EMA"].iloc[i] > bull["126EMA"].iloc[i]:
        Position.append("Long")
    else:
        Position.append("Short")

bull["Position"] = Position

bull["return"] = (bull["Adj Close"] - bull["Adj Close"].shift())/bull["Adj Close"].shift()
bull.dropna(inplace=True)

LongShort = [0]*bull.shape[0]
LongHold = [0]*bull.shape[0]

LongShort[0] = bull["Adj Close"].iloc[0]
LongShort[1] = bull["Adj Close"].iloc[0]
LongHold[0] = bull["Adj Close"].iloc[0]    
LongHold[1] = bull["Adj Close"].iloc[0]    

    
for i in range(0, bull.shape[0]-2):
    if bull["Position"].iloc[i] == "Long":
        LongShort[i+2] = LongShort[i+1]*(1+bull["return"][i+2])
    else:
        LongShort[i+2] = LongShort[i+1]/(1+bull["return"][i+2])
        
for i in range(0, bull.shape[0]-2):
    if bull["Position"].iloc[i] == "Long":
        LongHold[i+2] = LongHold[i+1]*(1+bull["return"][i+2])
    else:
        LongHold[i+2] = LongHold[i+1]

bull["LongShort"] = LongShort
bull["LongHold"] = LongHold
bull.drop(bull.tail(1).index,inplace=True)

fig = plt.figure(figsize=(20,12))
ax1 = plt.plot(bull["Adj Close"])
ax1 = plt.plot(bull["21EMA"])
ax1 = plt.plot(bull["126EMA"])
ax1 = plt.plot(bull["LongShort"], color="green")
ax1 = plt.plot(bull["LongHold"], color="brown")
ax1 = plt.title("INDIABULLS daily ajusted close and strategy from 2009 through 2018", fontsize=18)
ax1 = plt.xlabel("Date", fontsize=18)
ax1 = plt.ylabel("Price", fontsize=18)
ax1 = plt.legend(["Price", "EMA 21", "EMA 126", "21/126 day MACD long/short strategy", "21/126 day MACD long/cash strategy"],prop={"size":22}, loc="upper left")
plt.grid(True)
plt.show()

In [0]:
def strategies(df, days, MA=False, EMA=False, MACD=False):
    """
    Add selected moving averages to the DataFrame
    """
    if MA==True:
    # simple moving average
        df["MA"] = df["Adj Close"].rolling(window=days).mean()
    
    if EMA==True:
    # exponential moving average
        df["EMA"] = df["Adj Close"].ewm(span=days).mean()

    if MACD==True:
        # exponential moving average
        df["EMA_21"] = df["Adj Close"].ewm(span=21).mean()
        df["EMA_126"] = df["Adj Close"].ewm(span=126).mean()
            
    return df
    

def positions(df, MA=False, EMA=False, MACD=False):
    """
    calculates the positions we should hold each day according to each strategy
    """
    df_positions = pd.DataFrame(index=df.index)
    if MA==True:
    # Add position type for each day for the MA
        MA_Position = []    
        for i in range(0,df.shape[0]):
            if df["Adj Close"].iloc[i] > df["MA"].iloc[i]:
                MA_Position.append("Long")
            else:
                MA_Position.append("Short")
        df_positions["MA_Position"] = MA_Position
    
    if EMA==True:
    # Add position type for each day for the EMA
        EMA_Position = []
        for i in range(0,df.shape[0]):
            if df["Adj Close"].iloc[i] > df["EMA"].iloc[i]:
                EMA_Position.append("Long")
            else:
                EMA_Position.append("Short")
        df_positions["EMA_Position"] = EMA_Position
        
    if MACD==True:
    # Add position type for each day for the EMA
        MACD_Position = []
        for i in range(0,df.shape[0]):
            if df["EMA_21"].iloc[i] > df["EMA_126"].iloc[i]:
                MACD_Position.append("Long")
            else:
                MACD_Position.append("Short")
        df_positions["MACD_Position"] = MACD_Position
    
    return df_positions


def price_of_strategy(df, df_positions, LongHold=False, LongShort=False, MA=False, EMA=False, MACD=False):
    """
    given a DataFrame containing one or more position vectors, 
    Create price process of the strategies, adding the daily change if we are long
    subtracting if we are short.
    """
    df_price_of_strategy = pd.DataFrame(index=df_positions.index)
    df_price_of_strategy["asset price"] = df["Adj Close"]
    # long hold will long the strategy if condition is met, but instead of shorting it will
    # simply sell and wait for another entry point. A better version of this would be to
    # buy bonds instead of holding cash.
    if LongHold == True:
        if MA == True:
            LongHold = [0]*df.shape[0]
            LongHold[0] = df["Adj Close"].iloc[0]
            LongHold[1] = df["Adj Close"].iloc[0]
            for i in range(0, df_positions.shape[0]-2):
                if df_positions["MA_Position"].iloc[i] == "Long":
                    LongHold[i+2] = LongHold[i+1]*(1+df["return"][i+2])
                else:
                    LongHold[i+2] = LongHold[i+1]
            df_price_of_strategy["LongHold MA"] = LongHold
            
        if EMA == True:
            LongHold = [0]*df.shape[0]
            LongHold[0] = df["Adj Close"].iloc[0]
            LongHold[1] = df["Adj Close"].iloc[0]
            for i in range(0, df_positions.shape[0]-2):
                if df_positions["EMA_Position"].iloc[i] == "Long":
                    LongHold[i+2] = LongHold[i+1]*(1+df["return"][i+2])
                else:
                    LongHold[i+2] = LongHold[i+1]
            df_price_of_strategy["LongHold EMA"] = LongHold
            
        if MACD == True:
            LongHold = [0]*df.shape[0]
            LongHold[0] = df["Adj Close"].iloc[0]
            LongHold[1] = df["Adj Close"].iloc[0]
            for i in range(0, df_positions.shape[0]-2):
                if df_positions["MACD_Position"].iloc[i] == "Long":
                    LongHold[i+2] = LongHold[i+1]*(1+df["return"][i+2])
                else:
                    LongHold[i+2] = LongHold[i+1]
            df_price_of_strategy["LongHold MACD"] = LongHold
    
    
    if LongShort == True:
        if MA == True:
            LongShort = [0]*df.shape[0]
            LongShort[0] = df["Adj Close"].iloc[0]
            LongShort[1] = df["Adj Close"].iloc[0]
            for i in range(0, df.shape[0]-2):
                if df_positions["MA_Position"].iloc[i] == "Long":
                    LongShort[i+2] = LongShort[i+1]*(1+df["return"][i+2])
                else:
                    LongShort[i+2] = LongShort[i+1]/(1+df["return"][i+2])
            df_price_of_strategy["LongShort MA"] = LongShort
                
        if EMA == True:
            LongShort = [0]*df.shape[0]
            LongShort[0] = df["Adj Close"].iloc[0]
            LongShort[1] = df["Adj Close"].iloc[0]
            for i in range(0, df.shape[0]-2):
                if df_positions["EMA_Position"].iloc[i] == "Long":
                    LongShort[i+2] = LongShort[i+1]*(1+df["return"][i+2])
                else:
                    LongShort[i+2] = LongShort[i+1]/(1+df["return"][i+2])
            df_price_of_strategy["LongShort EMA"] = LongShort
            
        if MACD == True:
            LongShort = [0]*df.shape[0]
            LongShort[0] = df["Adj Close"].iloc[0]
            LongShort[1] = df["Adj Close"].iloc[0]
            for i in range(0, df.shape[0]-2):
                if df_positions["MACD_Position"].iloc[i] == "Long":
                    LongShort[i+2] = LongShort[i+1]*(1+df["return"][i+2])
                else:
                    LongShort[i+2] = LongShort[i+1]/(1+df["return"][i+2])
            df_price_of_strategy["LongShort MACD"] = LongShort
    
    return df_price_of_strategy


def strategy_returns(df, df_price_of_strategy):
    """
    input the price series of a strategy and output the return series
    """
    df_return_of_strategy = pd.DataFrame(index=df_price_of_strategy.index)
    cols = df_price_of_strategy.columns
    
    for priceSeries in cols:
        df_return_of_strategy[priceSeries] = (df_price_of_strategy[priceSeries]
                    - df_price_of_strategy[priceSeries].shift())/(df_price_of_strategy[priceSeries].shift())
    
    return df_return_of_strategy

In [0]:

def strategy(ticker, start, end, days, MA=False, EMA=False, MACD=False, LongHold=False, LongShort=False):
    """
    calculates the return on a moving average strategy of your choice
    """
    # load the data
    df = web.get_data_yahoo(f'{ticker}', 
                              start=start, 
                              end=end)
    # Daily return in the asset value
    df["return"] = (df["Adj Close"] - df["Adj Close"].shift())/df["Adj Close"].shift()
    
    # only keep the adjusted close and daily return
    df = df[["Adj Close", "return"]]
    
    # choose which moving averages we want
    df = strategies(df=df, days=100, MA=MA, EMA=EMA, MACD=MACD)
    
    # drop na's from the MA calculations
    df.dropna(inplace=True)
    
    # DataFrame of the positions we wish to hold
    df_positions = positions(df, MA=MA, EMA=EMA, MACD=MACD)

    #
    df_price_of_strategy = price_of_strategy(df, df_positions, LongHold=LongHold,
                                             LongShort=LongHold, MA=MA, EMA=EMA, MACD=MACD)
    df_price_of_strategy = df_price_of_strategy.drop(df_price_of_strategy.tail(2).index)

    # view summary statistics of the return series of the three strategies
    df_return_of_strategy = strategy_returns(df, df_price_of_strategy)
    df_return_of_strategy.dropna(inplace=True)
    #table = erk.summary_stats(df_return_of_strategy, ppp=252)
    return df, df_return_of_strategy, df_positions, df_price_of_strategy

def plot_strategy(ticker, start, end, df):
    """
    Plot the performance of the strategy vs the performance of the asset
    """
    fig = plt.figure(figsize=(20,15))
    ax1 = plt.plot(df)
    ax1 = plt.title("Comparing simple investment strategies for " +
                        ticker + " between " + start + " and " + end, fontsize=22)
    ax1 = plt.xlabel("Date", fontsize=18)
    ax1 = plt.ylabel("Price", fontsize=18)
    ax1 = plt.legend(list(df_price_of_strategy.columns),prop={"size":18}, loc="upper left")
    
    for legobj in ax1.legendHandles:
        legobj.set_linewidth(5.0)

    plt.grid(True)
    plt.show()


# Performance in Bull Markets¶
Apple from 2009 through 2018


S&P 500 from 2009 through 2018

In [0]:
start = datetime.date(2013, 7, 23)
end = datetime.date(2020, 3, 3)
ticker = "IBULHSGFIN.NS"
days = 100

df,  df_return_of_strategy, df_positions, df_price_of_strategy = strategy(
            ticker, start, end, days=days, MA=True, EMA=True, MACD=True, LongHold=True, LongShort=True)

plot_strategy(ticker=ticker, start=str(start), end=str(end), df=df_price_of_strategy)
